In [ ]:
import os
import csv
import yaml
import wandb
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.utils import to_categorical
from keras.layers import Dense
from keras.models import Sequential
from src import read_nz_file, read_jg_file, update_meta_data, split_df, aggregate_files, add_moving_window
from sklearn.model_selection import train_test_split
from wandb.keras import WandbCallback

processed data - intermediate step

> raw data

> clean data

> preprocess data : store it in DB (better compared to file format) 

    - data lake
    
    or
    
    - DB model based SQL (Nice to have but not required if we deceide to save the data as a Feather file)

> 

## 1. Load and clean raw data

files from SensorLog iOS app has in total over 70 colums and precision of 12 decimal figures. The output file is over 135 MB, which is too large for GitHub. GitHub restricts the file size, therefore these files from SensorLog must be cleaned.

Below summary of steps which is done only for iOS files:

- read raw data as csv files
- remove unnecessary columns (captured in list 'remove_cols' below)
- round to 6 decimal places to reduce the size of files
- output dataframe as csv
- upload the csv on GitHub

# 2. Meta data

The data on different data files is captured in meta dataframe below:
- file name
- user (nz or jg)
- activity (running/cycling/walking/sitting)
- pocket (in which pocket handy was during the activity)
- position_x
- position_y


# 3. View data

Sensor activity data is captured from 2 different Apps:
- SensorLog (iOS) by user 'nz'
- AndrioSensor (Andriod) by user 'jg'

**Response**: 'Acivity' with 4 classes: running/walking/cycling/sitting



In [ ]:
update_meta_data()

In [ ]:
def firstn():
    num = 0
    while True:
        yield num
        num += 1

k = firstn()
l = [next(k) for i in range(10)]
l

In [ ]:
meta = pd.read_csv('data/meta.csv')
meta = meta[::3]
meta

In [ ]:
# expects np array as input
def one_hot_encode(l):
    for i, v in enumerate(l):
        if v == 'sitting':
            l[i] = 0
        elif v == 'walking':
            l[i] = 1
        elif v == 'running':
            l[i] = 2
        elif v == 'cycling':
            l[i] = 3

    l = to_categorical(np.array(l))
    return l

In [ ]:
# create an aggregate of all files and output a list with the possible start positions within the dataframe. (lookback must always be contained within a single activity)
# saves this preprocessed file and list to the tmp folder
def preprocess_sequential_2(moving_window_seconds, hz, step_size, test_proportion = 0.2):

    # create empty data frames
    train = pd.DataFrame()
    test = pd.DataFrame()

    # create empty lists of start indexes
    train_indexes = []
    test_indexes = []

    for file, user, activity in zip(meta['file'], meta['user'], meta['activity']):
        if user == 'nz':
            df = read_nz_file(file, activity)
            df = df.drop(columns=['datetime'])

        elif user == 'jg':
            df = read_jg_file(file, activity)

        print(file, user, activity, df.shape)

        # split into train-test
        my_train_files, my_test_files = split_df(
            df, hz = hz, test_proportion = test_proportion, moving_window_size = moving_window_seconds
        )

        # aggregate data points (try moving average) transform to mean, sd, ...
        for i, (v_train, v_test) in enumerate(zip(my_train_files, my_test_files)):

            # save possible start indexes for training and test sequences
            lookback = hz * moving_window_seconds
            max_train_index = len(v_train) - 1
            max_test_index = len(v_test) - 1
            train_rows = np.arange(lookback, max_train_index, step_size)
            test_rows = np.arange(lookback, max_test_index, step_size)

            # append start indexes of sequences to total lists
            train_indexes += list(train_rows + len(train))
            test_indexes += list(test_rows + len(test))

            # append files to total file
            train = pd.concat([train, v_train])
            test = pd.concat([test, v_test])

    # split x and y
    y_train = train['y']
    x_train = train.drop(columns=['y', 'time_since_start(ms)'])
    y_test = test['y']
    x_test = test.drop(columns=['y', 'time_since_start(ms)'])

    # normalize data by training aggregates
    mean = x_train.mean(axis=0)
    x_train -= mean
    x_test -= mean
    std = x_train.std(axis=0)
    x_train /= std
    x_test /= std

    # one hot encode labels
    y_train = one_hot_encode(np.array(y_train))
    y_test = one_hot_encode(np.array(y_test))

    return x_train, x_test, y_train, y_test, train_indexes, test_indexes

In [ ]:
def save_sequential_preprocessing(
        X_train, X_test, y_train, y_test, train_indexes, test_indexes, folder: str, settings: str):

    if not os.path.exists(f'./tmp/{folder}'):
        os.mkdir(f'./tmp/{folder}')

    with open(f'tmp/{folder}/X_train.pickle', 'wb') as f: pickle.dump(X_train, f)
    with open(f'tmp/{folder}/X_test.pickle', 'wb') as f: pickle.dump(X_test, f)
    with open(f'tmp/{folder}/y_train.pickle', 'wb') as f: pickle.dump(y_train, f)
    with open(f'tmp/{folder}/y_test.pickle', 'wb') as f: pickle.dump(y_test, f)
    with open(f'tmp/{folder}/train_indexes.pickle', 'wb') as f: pickle.dump(train_indexes, f)
    with open(f'tmp/{folder}/test_indexes.pickle', 'wb') as f: pickle.dump(test_indexes, f)

    with open(rf'./tmp/{folder}/metadata.yaml', 'w') as file: yaml.dump(settings, file)

    print(f'Saved files to "./tmp/{folder}"')

In [ ]:
settings = {
    'MOVING_WINDOW_SIZE': 20,
    'HZ': 20,
    'STEP_SIZE': 20,
    'TEST_PROPORTION': 0.2,
    'AGGREGATION': "normal",
    'FEATURES': "all (mean & std)"
}

directory = f"sequential_{settings['HZ']}hz_{settings['MOVING_WINDOW_SIZE']}sec"
print(directory)

X_train, X_test, y_train, y_test, train_indexes, test_indexes = preprocess_sequential_2(
    moving_window_seconds = settings['MOVING_WINDOW_SIZE'],
    hz = settings['HZ'],
    step_size = settings['STEP_SIZE'],
    test_proportion = settings['TEST_PROPORTION']
)

save_sequential_preprocessing(X_train, X_test, y_train, y_test, train_indexes, test_indexes, directory, settings)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
# generator function that gets dataset and and yields tensors with a batch of sequences
# this function is used directly within keras


In [ ]:
def create_tensors(data, moving_window_seconds, hz, step_size):

    # split into x and y
    labels = data['y']
    data = data.drop(columns=['y'])

    # to numpy
    labels = labels.to_numpy()
    data = data.to_numpy()

    # select starter rows
    max_index = len(data) -1
    min_index = 0
    lookback = moving_window_seconds * hz

    i = min_index + lookback

    rows = np.arange(i, max_index, step_size)
    # print(rows)

    # create empty data structures
    samples = np.zeros((len(rows), lookback, data.shape[-1]))
    targets = []

    print(samples.shape)

    # add data for every starter row to data structures
    for j, row in enumerate(rows):
        indices = range(row - lookback, row)
        samples[j] = data[indices]
        targets.append(labels[row])

    return samples, targets

In [ ]:
def concat_tensor(total, new_part):
    if total is not None:
        total = np.concatenate((total, new_part))
    else:
        total = new_part

    return total

In [ ]:
def preprocess_sequential(moving_window_seconds, hz, step_size, test_proportion = 0.2):
    # create empty data frames
    x_train = None
    x_test = None
    y_train = []
    y_test = []

    for file, user, activity in zip(meta['file'], meta['user'], meta['activity']):
        if user == 'nz':
            df = read_nz_file(file, activity)
            df = df.drop(columns=['datetime'])

        elif user == 'jg':
            df = read_jg_file(file, activity)

        print(file, user, activity, df.shape)

        # create synthetic features

        # split into train-test
        my_train_files, my_test_files = split_df(
            df, hz = hz, test_proportion = test_proportion, moving_window_size = moving_window_seconds
        )

        # aggregate data points (try moving average) transform to mean, sd, ...
        for i, (v_train, v_test) in enumerate(zip(my_train_files, my_test_files)):

            x_train_next, y_train_next = create_tensors(v_train, moving_window_seconds, hz, step_size)
            x_test_next, y_test_next = create_tensors(v_test, moving_window_seconds, hz, step_size)

            x_train = concat_tensor(x_train, x_train_next)
            x_test = concat_tensor(x_test, x_test_next)
            y_train += y_train_next
            y_test += y_test_next

    # one hot encode labels
    y_train = one_hot_encode(y_train)
    y_test = one_hot_encode(y_test)

    # normalize data by training aggregates
    mean = x_train.mean(axis=0).mean(axis=0)
    x_train -= mean
    x_test -= mean
    std = x_train.std(axis=0).std(axis=0)
    x_train /= std
    x_test /= std

    return x_train, x_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = preprocess_sequential(20, 20, 20, test_proportion = 0.2)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:

def preprocess_data(moving_window_seconds, hz, step_size, agg_func ,test_proportion = 0.2):
    '''
    agg_func: aggregate function to apply eg add_moving_window or add_moving_window_2
    '''
    # create empty data frames
    train = pd.DataFrame()
    test = pd.DataFrame()

    for file, user, activity in zip(meta['file'], meta['user'], meta['activity']):
        if user == 'nz':
            df = read_nz_file(file, activity)

        elif user == 'jg':
            df = read_jg_file(file, activity)

        print(file, user, activity, df.shape)

        # create synthetic features

        # split into train-test
        my_train_files, my_test_files = split_df(
            df, hz = hz, test_proportion = test_proportion, moving_window_size = moving_window_seconds
        )

        # print(f'Train: {[len(i) for i in my_train_files]}')
        # print(f'Test: {[len(i) for i in my_test_files]}')

        # aggregate data points (try moving average) transform to mean, sd, ...
        for i, (v_train, v_test) in enumerate(zip(my_train_files, my_test_files)):
            # i.reset_index(drop = True)
            v_train = agg_func(
                v_train, hz_old_data = hz, seconds = moving_window_seconds, step_size = step_size
            )
            my_train_files[i] = v_train

            v_test = agg_func(
                v_test, hz_old_data = hz, seconds = moving_window_seconds, step_size = step_size
            )
            my_test_files[i] = v_test

        # print(f'Train: {[len(i) for i in my_train_files]}')
        # print(f'Test: {[len(i) for i in my_test_files]}')

        # append to train and test
        train = aggregate_files(my_train_files, train)
        test = aggregate_files(my_test_files, test)

    # X - y split for train and test data, shuffle data!?
    y_train = train['y'].to_frame()
    X_train = train.drop(columns=['y'])
    y_test = test['y'].to_frame()
    X_test = test.drop(columns=['y'])

    return X_train, X_test, y_train, y_test



In [ ]:
def save_preprocessing(X_train, X_test, y_train, y_test, folder: str, settings: str, file_type: str = 'parquet'):
    if not os.path.exists(f'./tmp/{folder}'):
        os.mkdir(f'./tmp/{folder}')

    if file_type == 'parquet':
        X_train.to_parquet(f'tmp/{folder}/X_train.parquet')
        X_test.to_parquet(f'tmp/{folder}/X_test.parquet')
        y_train.to_parquet(f'tmp/{folder}/y_train.parquet')
        y_test.to_parquet(f'tmp/{folder}/y_test.parquet')
    elif file_type == 'pickle':
        with open(f'tmp/{folder}/X_train.pickle', 'wb') as f: pickle.dump(X_train, f)
        with open(f'tmp/{folder}/X_test.pickle', 'wb') as f: pickle.dump(X_test, f)
        with open(f'tmp/{folder}/y_train.pickle', 'wb') as f: pickle.dump(y_train, f)
        with open(f'tmp/{folder}/y_test.pickle', 'wb') as f: pickle.dump(y_test, f)

    with open(rf'./tmp/{folder}/metadata.yaml', 'w') as file:
        yaml.dump(settings, file)

    print(f'Saved files to "./tmp/{folder}"')

In [ ]:
directory = 'basic_20hz_20sec'

settings = {
    'MOVING_WINDOW_SIZE': 20,
    'HZ': 20,
    'STEP_SIZE': 20,
    'TEST_PROPORTION': 0.2,
    'AGGREGATION': "normal",
    'FEATURES': "all (mean & std)",
    'PREPROCESSING': directory
}

X_train, X_test, y_train, y_test = preprocess_data(
    moving_window_seconds = settings['MOVING_WINDOW_SIZE'],
    hz = settings['HZ'],
    step_size = settings['STEP_SIZE'],
    test_proportion = settings['TEST_PROPORTION']
)

save_preprocessing(X_train, X_test, y_train, y_test, directory, settings)

In [ ]:
settings = {
    'MOVING_WINDOW_SIZE': 20,
    'HZ': 20,
    'STEP_SIZE': 20,
    'TEST_PROPORTION': 0.2,
    'AGGREGATION': "sequential",
    'FEATURES': "min, max, EucDist",
    'PREPROCESSING': directory
}

directory = f"NN_{settings['HZ']}hz_{settings['MOVING_WINDOW_SIZE']}sec"
print(directory)

X_train, X_test, y_train, y_test = preprocess_sequential(
    moving_window_seconds = settings['MOVING_WINDOW_SIZE'],
    hz = settings['HZ'],
    step_size = settings['STEP_SIZE'],
    test_proportion = settings['TEST_PROPORTION']
)

save_preprocessing(X_train, X_test, y_train, y_test, directory, settings, file_type='pickle')